In [ ]:
import pandas as pd
import numpy as np

from collections import defaultdict
from datetime import datetime
dct = defaultdict(list)
from google.colab import drive
!sudo apt-get update -y
!sudo apt-get install python3.8
!git clone https://github.com/awslabs/dgl-ke.git
%cd dgl-ke/python
!sudo python3 setup.py install
drive.mount('/content/gdrive/',force_remount=True)
%cd /content/gdrive/My\ Drive/

!pip3 install torch
!pip3 install dgl==0.4.3.post2
!pip install dglke==0.1.2


Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:11 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [1,825 kB]
Ign:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:13 https://developer.download

In [ ]:
import pandas as pd
import numpy as np

from collections import defaultdict
from datetime import datetime
dct = defaultdict(list)
from google.colab import drive
drive.mount('/content/gdrive/',force_remount=True)
%cd /content/gdrive/My\ Drive/

Mounted at /content/gdrive/
/content/gdrive/My Drive


In [ ]:
df = pd.read_csv('drug_repur/combined.csv')  
df.columns
df = df.drop(['SENTENCE','TRIPLE'],axis=1)
dct = defaultdict(list)
for index, row in df.iterrows():
  if row['SUBJECT_NAME'] == 'SUBJECT_NAME':
    #print(row)
    continue
  triple = (row['SUBJECT_NAME'],row['PREDICATE'],row['OBJECT_NAME'])
  edat = row['EDAT']
  dct[triple].append(datetime.strptime(edat, '%Y-%m-%d').date())
  
min_edat = {k: min(dct[k]) for k in dct}

In [5]:


print(len(df))

1672110


## Filter edge that less than mean freq

In [ ]:
df = pd.Series(min_edat).reset_index(drop=False)
df.columns = ['subject', 'relation', 'object', 'edat']
TH1 = datetime.strptime('2019-01-01', '%Y-%m-%d').date()
TH2 = datetime.strptime('2020-01-01', '%Y-%m-%d').date()
print(len(df))
df_train = df[df.edat <= TH1]
df_valid = df[(df.edat > TH1) & (df.edat <= TH2)]
df_test = df[df.edat > TH2]
dict1 = {}

def filter(df):
  for index, row in df.iterrows():
    name = (row['subject'],row['object'])
    value = row['relation']
    if name not in dict1:
      dict1[name] = {}
      dict1[name][value] = 1
    else:
      print(dict1[name])
      if value in dict1[name]:
        dict1[name][value] += 1
      else:
        dict1[name][value] = 1
  for i in dict1:
    if len(dict1[i])>1:
      sum1 = 0
      count = 0
      for j in dict1[i]:
        sum1 += dict1[i][j]
        count += 1
      avg = sum1/count
      for j in list(dict1[i]):
        if dict1[i][j] < avg:
          del dict1[i][j]
    return dict1
#filter(df_test)


791827


In [ ]:
object_set = set()
for index, row in df.iterrows():
  object_set.add(row['subject'])
  object_set.add(row['object'])

print(len(object_set))

112208


In [ ]:
temp = []
for key in df_train:
  sub, obj = key
  for rels in df_train[key]:
    temp.append([sub,obj,rels])
df_train = pd.DataFrame(temp)
df_train.columns = ['subject', 'relation', 'object']

dict1 = {}
dict2 = {}
dict3 = {}

def check_dup(df,df2):
  for index, row in df.iterrows():
    name = (row['subject'],row['object'],row['relation'])
    if name not in dict1:
      dict1[name] = True
    else:
      continue

  for index, row in df2.iterrows():
    name = (row['subject'],row['object'],row['relation'])
    if name not in dict1:
      dict2[name] = True
    else:
      continue

  return dict2


dict2 = check_dup(df_train,df_test)

## split test train

In [ ]:
df = pd.Series(min_edat).reset_index()
df.columns = ['subject', 'relation', 'object', 'edat']
TH1 = datetime.strptime('2012-01-01', '%Y-%m-%d').date()
TH2 = datetime.strptime('2020-03-11', '%Y-%m-%d').date()
df_train = df[df.edat <= TH1]
df_valid = df[(df.edat > TH1) & (df.edat <= TH2)]
df_test = df[df.edat > TH2]
dict1 = {}


## read train test

In [ ]:
df_train.to_csv('drug_repur/train.tsv', sep='\t', header=None, columns=['subject', 'relation', 'object'], index=False)
df_valid.to_csv('drug_repur/valid.tsv', sep='\t', header=None, columns=['subject', 'relation', 'object'], index=False)
df_test.to_csv('drug_repur/test.tsv', sep='\t', header=None, columns=['subject', 'relation', 'object'], index=False)

In [ ]:
import numpy as np
import itertools
import datetime

params = {
    "lr": [0.001, 0.01, 0.1],
    "neg_adv": [True, False],
    "hidden_dim": [50, 100, 250, 400],
    "norm": ['l1', 'l2'],
    "regularization_coef": [2e-6, 2e-8],
    "gamma": [1, 5, 10, 20],
}

def get_random_samples(params, n, replacement=False, seed=42):
    all_keys = params.keys()
    all_vals = [v for k,v in params.items()]
    combinations = list(itertools.product(*all_vals))
    
    rnd = np.random.RandomState(seed)
    res_list = [combinations[i] for i in rnd.choice(len(combinations), n, replace=replacement)]
    all_res = []
    
    for p in res_list:
        all_res.append( {k: p[i] for (i,k) in enumerate(all_keys)} )
        
    return all_res

n = 22
samples = get_random_samples(params, n)


DATA_PATH="drug_repur/"
SAVE_PATH="drug_repur/"
DATASET="NG"
FORMAT="raw_udd_hrt"

LOG_INTERVAL=1000
BATCH_SIZE=2070
BATCH_SIZE_EVAL=1020
NEG_SAMPLE_SIZE=30



import os

## TRANSE test1,0.569

In [ ]:


for idx, s in enumerate(samples):
    print(f"---TEST {idx + 1}---")
        
    model_name = f"TransE_{s['norm']}"
    neg_adv_flag = '-adv' if s['neg_adv'] else ''

    folder = '{}_{}_'.format(model_name, DATASET)
    n = len([x for x in os.listdir(SAVE_PATH) if x.startswith(folder)])
    folder += str(n)
    model_path = os.path.join(SAVE_PATH, folder)
    filename = f"./log/run_{idx + 1}_{folder}.txt"
    
    !DGLBACKEND=pytorch dglke_train --model_name {model_name} --data_path {DATA_PATH} --save_path {SAVE_PATH}  --dataset {DATASET}  --format {FORMAT} \
    --data_files train.tsv valid.tsv test.tsv \
    --log_interval {LOG_INTERVAL} --batch_size {BATCH_SIZE} --batch_size_eval {BATCH_SIZE_EVAL} --neg_sample_size {NEG_SAMPLE_SIZE} \
    --lr {s['lr']} {neg_adv_flag} --hidden_dim {s['hidden_dim']} -rc {s['regularization_coef']} -g {s['gamma']} \
    --gpu 0 --mix_cpu_gpu --max_step 10000
    
    !DGLBACKEND=pytorch dglke_eval --dataset {DATASET} --data_path {DATA_PATH} --data_files train.tsv valid.tsv test.tsv \
    --format {FORMAT} --model_name {model_name} --hidden_dim {s['hidden_dim']} \
    --gpu 0 --model_path {model_path} --batch_size_eval 1020 --neg_sample_size_eval 30 |& tee {filename}

---TEST 1---
Using backend: pytorch
Reading train triples....
Finished. Read 475343 train triples.
Reading valid triples....
Finished. Read 218113 valid triples.
Reading test triples....
Finished. Read 98371 test triples.
|Train|: 475343
/usr/local/lib/python3.7/dist-packages/dgl/base.py:25: UserWarning: multigraph will be deprecated.DGL will treat all graphs as multigraph in the future.
  warnings.warn(msg, warn_type)
Total initialize time 2.334 seconds
[proc 0][Train](1000/10000) average pos_loss: 0.6447117419242859
[proc 0][Train](1000/10000) average neg_loss: 0.4754200825449079
[proc 0][Train](1000/10000) average loss: 0.5600659118294716
[proc 0][Train](1000/10000) average regularization: 0.00015389497195653234
[proc 0][Train] 1000 steps take 7.945 seconds
[proc 0]sample: 1.673, forward: 2.679, backward: 1.386, update: 2.201
[proc 0][Train](2000/10000) average pos_loss: 0.6308773048520088
[proc 0][Train](2000/10000) average neg_loss: 0.39938225665688515
[proc 0][Train](2000/10000) 

In [ ]:

#TransE test1,0.569
!DGLBACKEND=pytorch dglke_eval --dataset NG --data_path drug_repur/ --data_files train.tsv valid.tsv test.tsv \
    --format raw_udd_hrt --model_name TransE_l2 --hidden_dim 400 \
    --gpu 0 --model_path drug_repur/TransE_l2_NG_14 --batch_size_eval 1020 --neg_sample_size_eval 30

Using backend: pytorch
Reading train triples....
Finished. Read 649924 train triples.
Reading valid triples....
Finished. Read 28872 valid triples.
Reading test triples....
Finished. Read 113031 test triples.
/usr/local/lib/python3.7/dist-packages/dgl/base.py:25: UserWarning: multigraph will be deprecated.DGL will treat all graphs as multigraph in the future.
  warnings.warn(msg, warn_type)
|valid|: 28872
|test|: 113031
[0]Test average MRR: 0.3824844417180361
[0]Test average MR: 10.526501782531195
[0]Test average HITS@1: 0.2780704099821747
[0]Test average HITS@3: 0.39745098039215687
[0]Test average HITS@10: 0.5778698752228164
Test takes 3.227 seconds


## DISTMULT TEST1 0.409

In [ ]:
for idx, s in enumerate(samples):
    print(f"---TEST {idx + 1}---")
        
    model_name = f"DistMult"
    neg_adv_flag = '-adv' if s['neg_adv'] else ''

    folder = '{}_{}_'.format(model_name, DATASET)
    n = len([x for x in os.listdir(SAVE_PATH) if x.startswith(folder)])
    folder += str(n)
    model_path = os.path.join(SAVE_PATH, folder)
    filename = f"log/run_{idx + 1}_{folder}.txt"
    
    !DGLBACKEND=pytorch dglke_train --model_name {model_name} --data_path {DATA_PATH} --save_path {SAVE_PATH}  --dataset {DATASET}  --format {FORMAT} \
    --data_files train.tsv valid.tsv test.tsv \
    --log_interval {LOG_INTERVAL} --batch_size {BATCH_SIZE} --batch_size_eval {BATCH_SIZE_EVAL} --neg_sample_size {NEG_SAMPLE_SIZE} \
    --lr {s['lr']} {neg_adv_flag} --hidden_dim {s['hidden_dim']} -rc {s['regularization_coef']} -g {s['gamma']} \
    --gpu 0 --mix_cpu_gpu --max_step 10000
    
    !DGLBACKEND=pytorch dglke_eval --dataset {DATASET} --data_path {DATA_PATH} --data_files train.tsv valid.tsv test.tsv \
    --format {FORMAT} --model_name {model_name} --hidden_dim {s['hidden_dim']} \
    --gpu 0 --model_path {model_path} --batch_size_eval 1020 --neg_sample_size_eval 30 |& tee {filename}

---TEST 1---
Reading train triples....
Finished. Read 784774 train triples.
Reading valid triples....
Finished. Read 218113 valid triples.
Reading test triples....
Finished. Read 98371 test triples.
|Train|: 784774
/usr/local/lib/python3.7/dist-packages/dgl/base.py:25: UserWarning: multigraph will be deprecated.DGL will treat all graphs as multigraph in the future.
  warnings.warn(msg, warn_type)
Total initialize time 3.910 seconds
[proc 0][Train](1000/10000) average pos_loss: 0.3471072804629803
[proc 0][Train](1000/10000) average neg_loss: 0.8002330640554428
[proc 0][Train](1000/10000) average loss: 0.5736701729297637
[proc 0][Train](1000/10000) average regularization: 0.0013651695819309282
[proc 0][Train] 1000 steps take 10.413 seconds
[proc 0]sample: 2.742, forward: 3.576, backward: 1.552, update: 2.535
[proc 0][Train](2000/10000) average pos_loss: 0.2934402098208666
[proc 0][Train](2000/10000) average neg_loss: 0.6163783984184266
[proc 0][Train](2000/10000) average loss: 0.45490930

In [ ]:
##
!DGLBACKEND=pytorch dglke_eval --dataset NG --data_path drug_repur/ --data_files train.tsv valid.tsv test.tsv \
    --format raw_udd_hrt --model_name DistMult --hidden_dim 50 \
    --gpu 0 --model_path drug_repur/DistMult_NG_1 --batch_size_eval 1020 --neg_sample_size_eval 30

Reading train triples....
Finished. Read 475343 train triples.
Reading valid triples....
Finished. Read 218113 valid triples.
Reading test triples....
Finished. Read 98371 test triples.
/usr/local/lib/python3.7/dist-packages/dgl/base.py:25: UserWarning: multigraph will be deprecated.DGL will treat all graphs as multigraph in the future.
  warnings.warn(msg, warn_type)
|valid|: 218113
|test|: 98371
[0]Test average MRR: 0.22539580206741092
[0]Test average MR: 14.582490808823529
[0]Test average HITS@1: 0.12781862745098038
[0]Test average HITS@3: 0.21464460784313724
[0]Test average HITS@10: 0.40190461601307187
Test takes 1.757 seconds


## ComplEx test0 0.464

In [ ]:
for idx, s in enumerate(samples):
    print(f"---TEST {idx + 1}---")
        
    model_name = f"ComplEx"
    neg_adv_flag = '-adv' if s['neg_adv'] else ''

    folder = '{}_{}_'.format(model_name, DATASET)
    n = len([x for x in os.listdir(SAVE_PATH) if x.startswith(folder)])
    folder += str(n)
    model_path = os.path.join(SAVE_PATH, folder)
    filename = f"drug_repur/log/run_{idx + 1}_{folder}.txt"
    
    !DGLBACKEND=pytorch dglke_train --model_name {model_name} --data_path {DATA_PATH} --save_path {SAVE_PATH}  --dataset {DATASET}  --format {FORMAT} \
    --data_files train.tsv valid.tsv test.tsv \
    --log_interval {LOG_INTERVAL} --batch_size {BATCH_SIZE} --batch_size_eval {BATCH_SIZE_EVAL} --neg_sample_size {NEG_SAMPLE_SIZE} \
    --lr {s['lr']} {neg_adv_flag} --hidden_dim {s['hidden_dim']} -rc {s['regularization_coef']} -g {s['gamma']} \
    --gpu 0 --mix_cpu_gpu --max_step 10000
    
    !DGLBACKEND=pytorch dglke_eval --dataset {DATASET} --data_path {DATA_PATH} --data_files train.tsv valid.tsv test.tsv \
    --format {FORMAT} --model_name {model_name} --hidden_dim {s['hidden_dim']} \
    --gpu 0 --model_path {model_path} --batch_size_eval 1020 --neg_sample_size_eval 30 |& tee {filename}

---TEST 1---
Using backend: pytorch
Reading train triples....
Finished. Read 649924 train triples.
Reading valid triples....
Finished. Read 28872 valid triples.
Reading test triples....
Finished. Read 113031 test triples.
|Train|: 649924
/usr/local/lib/python3.7/dist-packages/dgl/base.py:25: UserWarning: multigraph will be deprecated.DGL will treat all graphs as multigraph in the future.
  warnings.warn(msg, warn_type)
Total initialize time 6.101 seconds
[proc 0][Train](1000/10000) average pos_loss: 0.3262903085052967
[proc 0][Train](1000/10000) average neg_loss: 0.7837250455617905
[proc 0][Train](1000/10000) average loss: 0.5550076768100262
[proc 0][Train](1000/10000) average regularization: 0.001228301973596757
[proc 0][Train] 1000 steps take 7.852 seconds
[proc 0]sample: 1.605, forward: 2.677, backward: 1.466, update: 2.098
[proc 0][Train](2000/10000) average pos_loss: 0.3095175022482872
[proc 0][Train](2000/10000) average neg_loss: 0.5327767172753811
[proc 0][Train](2000/10000) ave

In [ ]:
##
!DGLBACKEND=pytorch dglke_eval --dataset NG --data_path drug_repur/ --data_files train.tsv valid.tsv test.tsv \
    --format raw_udd_hrt --model_name ComplEx --hidden_dim 50 \
    --gpu 0 --model_path drug_repur/ComplEx_NG_14 --batch_size_eval 1020 --neg_sample_size_eval 30

Using backend: pytorch
Reading train triples....
Finished. Read 649924 train triples.
Reading valid triples....
Finished. Read 28872 valid triples.
Reading test triples....
Finished. Read 113031 test triples.
/usr/local/lib/python3.7/dist-packages/dgl/base.py:25: UserWarning: multigraph will be deprecated.DGL will treat all graphs as multigraph in the future.
  warnings.warn(msg, warn_type)
|valid|: 28872
|test|: 113031
[0]Test average MRR: 0.28409624524762217
[0]Test average MR: 12.163489304812835
[0]Test average HITS@1: 0.1786007130124777
[0]Test average HITS@3: 0.2854055258467023
[0]Test average HITS@10: 0.47307486631016044
Test takes 2.769 seconds


## prediction

In [ ]:
!sudo apt-get update -y
!sudo apt-get install python3.8
!git clone https://github.com/awslabs/dgl-ke.git
%cd dgl-ke/python
!sudo python3 setup.py install
!pip3 install torch
!pip3 install dgl==0.4.3.post2

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:15 http://ppa.launchpad.net/graphics-drive

In [ ]:
import pandas as pd
import numpy as np

from collections import defaultdict
from datetime import datetime
dct = defaultdict(list)
from google.colab import drive
drive.mount('/content/gdrive/',force_remount=True)
%cd /content/gdrive/My\ Drive/

Mounted at /content/gdrive/
/content/gdrive/My Drive


In [ ]:
entity_emb = np.load('drug_repur/TransE_l2_NG_4/NG_TransE_l2_entity.npy')
rel_emb = np.load('drug_repur/TransE_l2_NG_4/NG_TransE_l2_relation.npy')
entities = pd.read_csv('drug_repur/entities.tsv', sep='\t', names=['id','entity'])
relations = pd.read_csv('drug_repur/relations.tsv', sep='\t', names=['id','rel'])


relation_list = relations.id.values.tolist()

entity2id = {}
id2entity = {}
for i in range(entities.shape[0]):
    entity2id[entities['entity'].values[i]] = i
    id2entity[i] = entities['entity'].values[i]
rel2id = {}
id2rel = {}
for i in range(relations.shape[0]):
    rel2id[relations['rel'].values[i]] = i
    id2rel[i] = relations['rel'].values[i]
    
with open('rel.list', 'w') as f:
    for item in relation_list:
        print(item)
        f.write("%s\n" % int(item))

#with open('head.list', 'w') as f:
 #   with open('./cui.list') as fh:
  #      for line in fh:
   #         cui = line.rstrip('\n')
    #        if cui in entity2id:
     #           f.write('%s\n' % entity2id[cui])
                
for idx, i in enumerate(relation_list):
    with open('./rel.list', 'w') as f:
        f.write("%s\n" % i)

tail = entities[(entities.entity.str.contains('C0002395'))]
tail_list = tail.id.values.tolist()

with open('tail.list', 'w') as f:
    for item in tail_list:
        f.write("%s\n" % item)


for idx, i in enumerate(relation_list):
    print(i)
    with open('./rel.list', 'w') as f:
        f.write("%s\n" % i)
    OUTFILE='./predictions/results_TransE_' + str(id2rel[i]) + '.txt'
    !DGLBACKEND=pytorch dglke_predict --model_path ./drug_repur/TransE_l2_NG_4/ --format '*_r_t' \
    --data_files rel.list tail.list --topK 10 --gpu 0 --exec_mode batch_head \
    --output {OUTFILE}

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
0
Using backend: pytorch
./drug_repur/TransE_l2_NG_4/config.json
{'dataset': 'NG', 'model': 'TransE_l2', 'emb_size': 250, 'max_train_step': 10000, 'batch_size': 2070, 'neg_sample_size': 30, 'lr': 0.01, 'gamma': 10.0, 'double_ent': False, 'double_rel': False, 'neg_adversarial_sampling': False, 'adversarial_temperature': 1.0, 'regularization_coef': 2e-08, 'regularization_norm': 3, 'emap_file': 'entities.tsv', 'rmap_file': 'relations.tsv'}
Traceback (most recent call last):
  File "/usr/local/bin/dglke_predict", line 33, in <module>
    sys.exit(load_entry_point('dglke==0.1.0.dev0', 'console_scripts', 'dglke_predict')())
  File "/usr/local/lib/python3.7/dist-packages/dglke/infer_score.py", line 216, in main
    result = model.topK(head, rel, tail, args.exec_mode, args.topK)
  File "/usr/local/lib/python3.7/dist-packages/dglke/models/infer.py", line 160, in topK
    raw_score = self.model.score(F.unsqueeze(head[i], 0), rel, tail)
  File "/usr/local/lib/py

In [ ]:
id2rel

{0: 'AFFECTS',
 1: 'CAUSES',
 2: 'INTERACTS_WITH',
 3: 'ASSOCIATED_WITH',
 4: 'DISRUPTS',
 5: 'AUGMENTS',
 6: 'STIMULATES',
 7: 'INHIBITS',
 8: 'COEXISTS_WITH',
 9: 'PREDISPOSES',
 10: 'PRODUCES',
 11: 'PREVENTS',
 12: 'TREATS',
 13: 'MANIFESTATION_OF',
 14: 'COMPLICATES'}